# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115179e+02     1.716351e+00
 * time: 0.08841800689697266
     1     1.009936e+01     1.008122e+00
 * time: 0.9646458625793457
     2    -1.182187e+01     1.115594e+00
 * time: 1.1079699993133545
     3    -3.404592e+01     9.234779e-01
 * time: 1.2557790279388428
     4    -4.741256e+01     6.969890e-01
 * time: 1.3903608322143555
     5    -5.684468e+01     2.642941e-01
 * time: 1.532270908355713
     6    -5.963077e+01     1.907678e-01
 * time: 1.6446399688720703
     7    -6.077207e+01     7.092645e-02
 * time: 1.7575969696044922
     8    -6.129573e+01     4.908872e-02
 * time: 1.8621678352355957
     9    -6.160735e+01     3.688814e-02
 * time: 1.9752588272094727
    10    -6.177529e+01     3.127654e-02
 * time: 2.0893189907073975
    11    -6.193558e+01     2.545849e-02
 * time: 2.196645975112915
    12    -6.203111e+01     1.715021e-02
 * time: 2.310851812362671
    13    -6.208508e+01     1.542355e-02
 * time: 2.424458980560

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557685
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666461345
